In [1]:
from lib.outliers import Runner
from datetime import date

In [2]:
from_date = date(year=2021,month=6,day=1)
to_date = date(year=2021,month=12,day=1)
r = Runner(from_date=from_date,to_date=to_date,n_outliers=10,entities=["practice","ccg","pcn","stp"],force_rebuild=False,n_jobs=16)

In [3]:
r.run()

Downloading: 100%|██████████| 1/1 [00:00<00:00,  7.63rows/s]


/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/home/app/notebook/lib/outliers.py:645: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set_xlim(lower_limit, upper_limit)
/home/app/notebook/lib/outliers.py:645: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set_xlim(lower_limit, upper_limit)
/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/seaborn/distributions.py:288: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/home/app/notebook/lib/outliers.py:645: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set_xlim(lower_limit, upper_limit)
/hom